#Sentiment Analysis

##Objective: To Design a model to extract, identify and characterize sentitment of the Tweet

###Visualizing Global Impact of CoronaVirus

####Extracting country and US state data

In [5]:
%sh
wget https://raw.githubusercontent.com/harshdesai7/Titanic/master/Country.csv
wget https://raw.githubusercontent.com/harshdesai7/Titanic/master/state.csv

--2020-05-03 14:36:51-- https://raw.githubusercontent.com/harshdesai7/Titanic/master/Country.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4366 (4.3K) [text/plain]
Saving to: ‘Country.csv’

 0K .... 100% 12.7M=0s

2020-05-03 14:36:51 (12.7 MB/s) - ‘Country.csv’ saved [4366/4366]

--2020-05-03 14:36:51-- https://raw.githubusercontent.com/harshdesai7/Titanic/master/state.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 728 [text/plain]
Saving to: ‘state.csv’

 0K 100% 99.4M=0s

2020-05-03 14:36:51 (99.4 MB/s) - ‘state.csv’ saved [728/728]

####Getting Live Data from API and binding it to Spark DataFrame

In [7]:
#consuming the data provided by api.covid19api.com in json and converting it into Spark DataFrame
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
request = requests.get("https://api.covid19api.com/summary")
#request = requests.get("https://api.thevirustracker.com/free-api?global=stats")
response = json.loads(request.text)
dfOverallStats=spark.createDataFrame(json_normalize(response['Global']))
display(dfOverallStats)

NewConfirmed,NewDeaths,NewRecovered,TotalConfirmed,TotalDeaths,TotalRecovered
82251,5007,40226,3230493,217954,1039749


In [8]:
#consuming the data provided by api.covid19api.com in json and converting it into Spark DataFrame
request = requests.get("https://api.covid19api.com/summary")
response = json.loads(request.text)
dfCountryStats=spark.createDataFrame(json_normalize(response['Countries']))
dfCountryStats.count()
#display(dfCountryStats)

Out[8]: 244

In [9]:
#displaying country wise total confirmed cases using select and where clause to have latest data
from pyspark.sql.functions import col
dfToday=dfCountryStats.select('Country','TotalConfirmed').where(col('Date').like("%2020%"))
display(dfToday)

Country,TotalConfirmed
ALA Aland Islands,0
Afghanistan,2469
Albania,789
Algeria,4295
American Samoa,0
Andorra,747
Angola,35
Anguilla,0
Antarctica,0
Antigua and Barbuda,25


####Total cases over a period of time globally

In [11]:
#Extracting information from API and displaying Bar Chart Visualization of total cases against count dateswise 
response = requests.get("https://thevirustracker.com/timeline/map-data.json")
todos = json.loads(response.text)
dfCountryData = spark.createDataFrame(json_normalize(todos['data']))
display(dfCountryData)
dfCountryData = dfCountryData.withColumn('cases',  dfCountryData['cases'].cast('int'))
display(dfCountryData.select("date","cases"))

date,cases
3/02/20,1
3/03/20,1
3/04/20,1
3/05/20,1
3/06/20,1
3/07/20,1
3/08/20,1
3/09/20,1
3/10/20,1
3/11/20,1


####Recovered vs Total Cases

In [13]:
#displaying Area Chart Visualization of total confimed cases cases and recovered cases against count dateswise
dfCountryData = dfCountryData.withColumn('recovered',  dfCountryData['recovered'].cast('int'))
display(dfCountryData.select("date","cases","recovered"))

date,cases,recovered
3/02/20,1,0
3/03/20,1,0
3/04/20,1,0
3/05/20,1,0
3/06/20,1,0
3/07/20,1,0
3/08/20,1,0
3/09/20,1,0
3/10/20,1,0
3/11/20,1,0


In [14]:
#Reading the downloaded csv file for Mapping State and Country with the API Data
dfCountries = spark.read.csv('file:/databricks/driver/Country.csv',header=True)
#display(dfCountries)
dfstates = spark.read.csv('file:/databricks/driver/state.csv',header=True)
#display(dfstates)

####World Map showing affected countries
######To see individual country data, hover over the country

In [16]:
#Displaying Visualization of World wide total count of corona virus cases. Hover over a country to know it's total count
worldRDD=dfCountries.join(dfToday, dfCountries["Country"] == dfToday['Country']).select('Code','TotalConfirmed')
worldRDD=worldRDD.withColumnRenamed("Code", "country")
worldRDD=worldRDD.withColumnRenamed("TotalConfirmed", "value")
display(worldRDD)

country,value
AFG,2469
ALB,789
DZA,4295
ASM,0
AND,747
AGO,35
AIA,0
ATA,0
ATG,25
ARG,4681


In [17]:
#Extracting US state wise count from API in json and converting it into DataFrame 
from pyspark.sql.functions import col, expr
request = requests.get("https://api.covid19api.com/live/country/us/status/confirmed")
response = json.loads(request.text)
data=spark.createDataFrame(response)
df1=data.orderBy("Date", ascending=False)
#display(df1)


/databricks/spark/python/pyspark/sql/session.py:355: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
 warnings.warn("inferring schema from dict is deprecated,"

In [18]:
display(df1)

Active,City,CityCode,Confirmed,Country,CountryCode,Date,Deaths,Lat,Lon,Province,Recovered
972,,,1023,United States of America,US,2020-04-28T00:00:00Z,51,44.69,-69.38,Maine,0
3525,,,3811,United States of America,US,2020-04-28T00:00:00Z,286,45.69,-93.9,Minnesota,0
6311,,,6539,United States of America,US,2020-04-28T00:00:00Z,228,32.32,-86.9,Alabama,0
9734,,,9918,United States of America,US,2020-04-28T00:00:00Z,184,35.75,-86.69,Tennessee,0
4195,,,4236,United States of America,US,2020-04-28T00:00:00Z,41,40.15,-111.86,Utah,0
25328,,,27068,United States of America,US,2020-04-28T00:00:00Z,1740,31.17,-91.87,Louisiana,0
513,,,520,United States of America,US,2020-04-28T00:00:00Z,7,42.76,-107.3,Wyoming,0
12,,,14,United States of America,US,2020-04-28T00:00:00Z,2,15.1,145.67,Northern Mariana Islands,0
24655,,,25321,United States of America,US,2020-04-28T00:00:00Z,666,31.05,-97.56,Texas,0
3707,,,3892,United States of America,US,2020-04-28T00:00:00Z,185,38.9,-77.03,District of Columbia,0


In [19]:
#sorting the data in datewise descending to filter latest count per state
currdate=(df1.select('*').orderBy('Date','Province',ascending=False).limit(56))
display(currdate)

Active,City,CityCode,Confirmed,Country,CountryCode,Date,Deaths,Lat,Lon,Province,Recovered
513,,,520,United States of America,US,2020-04-28T00:00:00Z,7,42.76,-107.3,Wyoming,0
5800,,,6081,United States of America,US,2020-04-28T00:00:00Z,281,44.27,-89.62,Wisconsin,0
1029,,,1063,United States of America,US,2020-04-28T00:00:00Z,34,38.49,-80.95,West Virginia,0
13078,,,13538,United States of America,US,2020-04-28T00:00:00Z,460,37.77,-78.17,Virginia,0
53,,,57,United States of America,US,2020-04-28T00:00:00Z,4,18.34,-64.9,Virgin Islands,0
808,,,855,United States of America,US,2020-04-28T00:00:00Z,47,44.05,-72.71,Vermont,0
4195,,,4236,United States of America,US,2020-04-28T00:00:00Z,41,40.15,-111.86,Utah,0
24655,,,25321,United States of America,US,2020-04-28T00:00:00Z,666,31.05,-97.56,Texas,0
9734,,,9918,United States of America,US,2020-04-28T00:00:00Z,184,35.75,-86.69,Tennessee,0
2233,,,2244,United States of America,US,2020-04-28T00:00:00Z,11,44.3,-99.44,South Dakota,0


####US Map showing affected states
######To see individual states data, hover over the states

In [21]:
#Displaying Visualization of US state wide total count of corona virus cases. Hover over a state to know it's total count
stateRDD=dfstates.join(currdate, dfstates["State"] == currdate['Province']).select('Code','Confirmed')
stateRDD=stateRDD.withColumnRenamed("Code", "state")
stateRDD=stateRDD.withColumnRenamed("Confirmed", "value")
display(stateRDD)

state,value
WY,520
WI,6081
WV,1063
VA,13538
VT,855
UT,4236
TX,25321
TN,9918
SD,2244
SC,5613


###Tableau Visualizations

####Please <a href="https://public.tableau.com/views/State_15881828842450/Dashboard2?:display_count=y&publish=yes&:origin=viz_share_link" target=_blank> click here </a> to view the visualizations performed in Tableau

####Streaming Tweets using Tweepy API

In [25]:
#installing tweepy in order to use Tweepy API

In [26]:
%sh
pip install tweepy

Collecting tweepy
 Downloading https://files.pythonhosted.org/packages/36/1b/2bd38043d22ade352fc3d3902cf30ce0e2f4bf285be3b304a2782a767aec/tweepy-3.8.0-py2.py3-none-any.whl
Requirement already satisfied: requests>=2.11.1 in /databricks/python3/lib/python3.7/site-packages (from tweepy) (2.21.0)
Requirement already satisfied: six>=1.10.0 in /databricks/python3/lib/python3.7/site-packages (from tweepy) (1.12.0)
Collecting requests-oauthlib>=0.7.0 (from tweepy)
 Downloading https://files.pythonhosted.org/packages/a3/12/b92740d845ab62ea4edf04d2f4164d82532b5a0b03836d4d4e71c6f3d379/requests_oauthlib-1.3.0-py2.py3-none-any.whl
Requirement already satisfied: PySocks>=1.5.7 in /databricks/python3/lib/python3.7/site-packages (from tweepy) (1.6.8)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.11.1->tweepy) (3.0.4)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.11.1->tweepy) (2019.3.9)
Requirement already satisfied: urllib3<1.25,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.11.1->tweepy) (1.24.1)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests>=2.11.1->tweepy) (2.8)
Collecting oauthlib>=3.0.0 (from requests-oauthlib>=0.7.0->tweepy)
 Downloading https://files.pythonhosted.org/packages/05/57/ce2e7a8fa7c0afb54a0581b14a65b56e62b5759dbc98e80627142b8a3704/oauthlib-3.1.0-py2.py3-none-any.whl (147kB)
Installing collected packages: oauthlib, requests-oauthlib, tweepy
Successfully installed oauthlib-3.1.0 requests-oauthlib-1.3.0 tweepy-3.8.0
You are using pip version 19.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

####Authenticating with Twitter Credentials using OAuth

In [28]:
#After registering on Developer Twitter website and retreiving the Keys
#We are validating it using OAuth Authentication to gain access of the tweets provided by the tweepy API
import tweepy
import csv
import ssl
import time
import socket
from tweepy.auth import OAuthHandler
from tweepy import Stream
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError
consumer_key = 'EkJJSAt55n5v1vcUo0jFg'
consumer_secret = 'o8sE5KA9NH5blHjIPBnBOiUz48D9buo3UmBMD0Js'
access_token = '59322976-kRknlhcWTcnfs6dk4aKXRlhnWD0P7K3mD2h44xt7d'
access_token_secret = 'u7tQzPZHbMrOjkAZYrNPx1FElqJejSqFWcCEwygr9dPvM'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

####Filtering Tweets relating to Coronavirus

In [30]:
#Mentioning the Keywords in order to filter out Keys related to CoronaVirus
import csv
import ssl
import time
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError
# Setting up the keywords, hashtag or mentions we want to listen
keywords = ["#Covid19", "Covid19","Coronavirus"]

# Set the name for CSV file where the tweets will be saved
filename = "myfile"

####Extracting Filtered Tweets

#####Starts extracting Tweet, Run for 10 minutes to extract approximately 2000 Tweets. Have to manually stop execution to stop the streaming

In [33]:

class StreamListener(tweepy.StreamListener):

    def on_status(self, status):

        try:
            # saves the tweet object
            tweet_object = status

            # Checks if its a extended tweet (>140 characters)
            if 'extended_tweet' in tweet_object._json:
                tweet = tweet_object.extended_tweet['full_text']
            else:
                tweet = tweet_object.text

#Convert all named and numeric character references (e.g. &gt;, &#62;, &#x3e;) in the strings to their corresponding Unicode characters
            tweet = (tweet.replace('&amp;', '&').replace('&lt;', '<')
                     .replace('&gt;', '>').replace('&quot;', '"')
                     .replace('&#39;', "'").replace(';', " ")
                     .replace(r'\u', " "))

#Save the keyword that matches the stream
            keyword_matches = []
            for word in keywords:
                if word.lower() in tweet.lower():
                    keyword_matches.extend([word])

            keywords_strings = ", ".join(str(x) for x in keyword_matches)

#Save other information from the tweet
            user = status.author.screen_name
            timeTweet = status.created_at
            source = status.source
            tweetId = status.id
            tweetUrl = "https://twitter.com/statuses/" + str(tweetId)

#Exclude retweets, too many mentions and too many hashtags
            if not any((('RT @' in tweet, 'RT' in tweet,
                       tweet.count('@') >= 2, tweet.count('#') >= 3))):

#Saves the tweet information in a new row of the CSV file
                writer.writerow([tweet, keywords_strings, timeTweet,
                                user, source, tweetId, tweetUrl])

        except Exception as e:
            print('Encountered Exception:', e)
            pass


def work():

#Opening a CSV file to save the gathered tweets
    with open(filename+".csv", 'w') as file:
        global writer
        writer = csv.writer(file)

#Add a header row to the CSV
        writer.writerow(["Tweet", "Matched Keywords", "Date", "User",
                        "Source", "Tweet ID", "Tweet URL"])

#Initializing the twitter Stream and also trying to convert other languages to english
        try:
            streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
            streamingAPI.filter(languages=["en"],track=keywords)

#Stop temporarily when hitting Twitter rate Limit
        except tweepy.RateLimitError:
            print("RateLimitError...waiting ~15 minutes to continue")
            time.sleep(1001)
            streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
            streamingAPI.filter(languages=["en"],track=keywords)

#Stop temporarily when getting a timeout or connection error
        except (Timeout, ssl.SSLError, ReadTimeoutError,
                ConnectionError) as exc:
            print("Timeout/connection error...waiting ~15 minutes to continue")
            time.sleep(1001)
            streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
            streamingAPI.filter(languages=["en"],track=keywords)

#Stop temporarily when getting other errors
        except tweepy.TweepError as e:
            if 'Failed to send request:' in e.reason:
                print("Time out error caught.")
                time.sleep(1001)
                streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
                streamingAPI.filter(languages=["en"],track=keywords)
            else:
                print("Other error with this user...passing")
                pass


if __name__ == '__main__':

    work()

In [34]:
%sh
ls

conf
Country.csv
derby.log
eventlogs
ganglia
logs
myfile.csv
state.csv

In [35]:
#reading the streamed tweets from csv file
from pyspark.sql.functions import col
dfTweets = spark.read.csv('file:///databricks/driver/myfile.csv',inferSchema='true',header='true')
dfTweets.count()


Out[24]: 2438

###Data Cleaning

In [37]:
display(dfTweets)

Tweet,Matched Keywords,Date,User,Source,Tweet ID,Tweet URL
"#coronavirus update in #US. New cases confirmed 11, new deaths 1. Total cases in US confirmed 1164007, recovered 175382, deaths 67552. +0.39% daily cases increase, 4577 reported today UTC time.",Coronavirus,2020-05-03 14:48:39,CoronaSpread,CoronaLiveUpdates,1256958855927541760,https://twitter.com/statuses/1256958855927541760
"Be that as it may, the restructuring of CDC last year, that eliminated positions to oversee emerging viruses in China, worsened our ability to prepare.",null,2020-05-03 14:48:39,stenwick,Twitter for iPhone,1256958856149622784,https://twitter.com/statuses/1256958856149622784
"Top US, Somalia diplomats thank Turkey for support, medical aid amid coronavirus pandemic https://t.co/gvyaqinHfV via @DailySabah",Coronavirus,2020-05-03 14:48:39,Orgetorix,Twitter for iPhone,1256958857240346624,https://twitter.com/statuses/1256958857240346624
"I wonder If China were hard beaten by this virus, will the world lash on China. https://t.co/JoZNA6R4x7",null,2020-05-03 14:48:39,weiwen,Twitter for iPhone,1256958857491779585,https://twitter.com/statuses/1256958857491779585
"@HillBeverlyhill YOU HAD FORGOTTEN TO LIST ONE OTHER ITEM! TRUMPS A MURDER HAVING WELL OVER 65,000 AMERICAN CITIZENS BLOOD ON HIS HANDS.TRUMP KNEW IN LATE DEC EARLY JAN ABOUT COVID19 & CHOSE TO IGNORE IT HE THEN CALLED COVID19 DEMOCRATIC HOAX, THEN HE LIED ABOUT COVID19 & FINALLY MISLEAD US!",Covid19,2020-05-03 14:48:39,LLochaber,Twitter for iPhone,1256958857441447939,https://twitter.com/statuses/1256958857441447939
This is why we need Joe Biden #GoJoe,null,2020-05-03 14:48:39,ActivePhysique,Twitter Web App,1256958857525497858,https://twitter.com/statuses/1256958857525497858
should we laugh,null,2020-05-03 14:48:39,Kissan1998,Twitter Web App,1256958857882095617,https://twitter.com/statuses/1256958857882095617
"@realDonaldTrump FakeNews, FB, Twitter R liars R the Enemies of the Truth & the American People! FakeNews spew hate-filled Democratic Party Plantation Propaganda! Americans R being lied 2 by the FakeNews about President Trump, cures 4 COVID19, & Democrats’ & Bureaucracy corruption! Woke!",Covid19,2020-05-03 14:48:39,fmiller26,Twitter for iPhone,1256958858343460864,https://twitter.com/statuses/1256958858343460864
"Coronavirus Outbreak LIVE Updates: Gujarat reports 374 new infections, 28 fatalities in 24 hours https://t.co/fmQTQLaB8U",Coronavirus,2020-05-03 14:48:40,iyer_sk,Twitter Web Client,1256958860339712000,https://twitter.com/statuses/1256958860339712000
WHO: US just reported deadliest day for coronavirus https://t.co/qUPb9zU4PB,Coronavirus,2020-05-03 14:48:40,POTUS45stinks,Twitter for iPhone,1256958860402634753,https://twitter.com/statuses/1256958860402634753


####Converting DataFrame to List

In [39]:
#taking tweet column as list
from pyspark.sql.functions import col
tweetslist = dfTweets.select(col("Tweet")).collect()
tweetslist

Out[26]: [Row(Tweet='#coronavirus update in #US. New cases confirmed 11, new deaths 1. Total cases in US confirmed 1164007, recovered 175382, deaths 67552. +0.39% daily cases increase, 4577 reported today UTC time.'),
 Row(Tweet='Be that as it may, the restructuring of CDC last year, that eliminated positions to oversee emerging viruses in China, worsened our ability to prepare.'),
 Row(Tweet='Top US, Somalia diplomats thank Turkey for support, medical aid amid coronavirus pandemic https://t.co/gvyaqinHfV via @DailySabah'),
 Row(Tweet='I wonder If China were hard beaten by this virus, will the world lash on China. https://t.co/JoZNA6R4x7'),
 Row(Tweet='@HillBeverlyhill YOU HAD FORGOTTEN TO LIST ONE OTHER ITEM! TRUMPS A MURDER HAVING WELL OVER 65,000 AMERICAN CITIZENS BLOOD ON HIS HANDS.TRUMP KNEW IN LATE DEC EARLY JAN ABOUT COVID19 & CHOSE TO IGNORE IT HE THEN CALLED COVID19 DEMOCRATIC HOAX, THEN HE LIED ABOUT COVID19 & FINALLY MISLEAD US!'),
 Row(Tweet='This is why we need Joe Biden #GoJoe'),
 Row(Tweet='should we laugh'),
 Row(Tweet='@realDonaldTrump FakeNews, FB, Twitter R liars R the Enemies of the Truth & the American People! FakeNews spew hate-filled Democratic Party Plantation Propaganda! Americans R being lied 2 by the FakeNews about President Trump, cures 4 COVID19, & Democrats’ & Bureaucracy corruption! Woke!'),
 Row(Tweet='Coronavirus Outbreak LIVE Updates: Gujarat reports 374 new infections, 28 fatalities in 24 hours https://t.co/fmQTQLaB8U'),
 Row(Tweet='WHO: US just reported deadliest day for coronavirus https://t.co/qUPb9zU4PB'),
 Row(Tweet='@rindyaird These young people will likely not have severe #COVID, because they are young and healthy. \n'),
 Row(Tweet='However'),
 Row(Tweet='Here’s the close-up video.. '),
 Row(Tweet='https://t.co/GrDhoUh5A0"'),
 Row(Tweet='@Rscube3 Or you can say coronavirus and non coronavirus patients'),
 Row(Tweet='BRUSSELS — Australia has called for an inquiry into the origin of the virus. Germany and Britain are hesitating anew about inviting in the Chinese tech giant Huawei. President Trump has blamed China for the contagion and is seeking to punish it. https://t.co/l0wWFXGQo0'),
 Row(Tweet='Adidas is using AI and 3D printing to cope with a plunge in sales during the coronavirus pandemic https://t.co/KtVHRYCaYi via @businessinsider'),
 Row(Tweet='"What was your COVID19 Crisis turning point? For me it\'s still Feb. 25th. CDC conf call. Dr. Nancy Messonnier: ""We are asking the American public to work with us to prepare in the expectation that this could be bad...It’s not so much a question of if this will happen [but] when."""'),
 Row(Tweet='Itna waist of money kyo warriors to simple flowers se bhi khush hojate'),
 Row(Tweet='No thanks......you can keep it!'),
 Row(Tweet='It is ridiculous to say America should reopen economy, go to work, take Covid-19 risk, when the leaders of the country are safe at home and not practicing what they are preaching.'),
 Row(Tweet='Coronavirus Update: San #Francisco Hotels Struggling During COVID-19 Shutdown - May 3 @ 10:45 AM ET https://t.co/mJvZWjpZVm'),
 Row(Tweet='COVID19 lockdown, a case study for environmental awareness - YouTube - https://t.co/trCioMGH9w'),
 Row(Tweet='Absolute fucking tosser'),
 Row(Tweet='The War Between Middle Schoolers and Their Parents Ends Now https://t.co/TO6BdnD7mu'),
 Row(Tweet='The @NCLC4consumers estimates that 1 in 3 Americans have debt in collections. \n'),
 Row(Tweet='“Federal law requires that debtors are left with at least $217.50 a week in take-home pay—for a family of four'),
 Row(Tweet='https://t.co/NLn0UjGddT"'),
 Row(Tweet='We love to see it🤣🤣🤣🤣'),
 Row(Tweet='@realDonaldTrump Then they impeach you.\n'),
 Row(Tweet='Then you botch the Covid19 response.'),
 Row(Tweet='Then you lose to a brain damaged Democrat."'),
 Row(Tweet='Wuhan Lab released the Virus and wants the patent a cure Remdesivir. Think about who is touting this drug... https://t.co/KR2WA1koZs'),
 Row(Tweet='this has been wildly derailed by coro

####Cleaning the Tweets Using Regular Expression

In [41]:
#UDF for Removing Patterns
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt

In [42]:
import numpy as np
import re
rem_link = []
list1=[]
for i in range(0,len(tweetslist)-1):
  # Using Regex to remove row from tweets
  list1.append(remove_pattern(str(tweetslist[i]), "Row[A-Za-z0-9./]*"))
list2 = []
for i in range(0,len(tweetslist)-1):
   # Using Regex to remove the word tweet from tweets
  list2.append(remove_pattern(str(list1[i]), "Tweet[A-Za-z0-9./]*"))
list2
for i in range(0,len(list2)-1):
   # Removing url's from the tweets list
  rem_link.append(remove_pattern(str(list2[i]), "https?://[A-Za-z0-9./]*"))
rem_retweet = []
for i in range(0,len(rem_link)-1):
   # Using Regex to remove retweets
  if not rem_link[i].startswith('RT'):
    rem_retweet.append(rem_link[i])
rem_specialChar = []
for i in range(0,len(rem_retweet)-1):
  #elem = re.sub( '[^a-z0-9A-Z]', ' ', cleaned_list1[i])
  rem_specialChar.append(re.sub( '[^a-z0-9A-Z]', ' ', rem_retweet[i]))
clean = []
for i in range(0,len(rem_specialChar)-1):
  clean.append(re.sub( 's/^\s+|\s+$|\s+(?=\s)//g', ' ', rem_specialChar[i]))
cleaned_tweets = clean
for x in range(len(cleaned_tweets)):
  #stripping the whitespace
  cleaned_tweets[x]=cleaned_tweets[x].strip()
cleaned_tweets

Out[28]: ['coronavirus update in US New cases confirmed 11 new deaths 1 Total cases in US confirmed 1164007 recovered 175382 deaths 67552 0 39 daily cases increase 4577 reported today UTC time',
 'Be that as it may the restructuring of CDC last year that eliminated positions to oversee emerging viruses in China worsened our ability to prepare',
 'Top US Somalia diplomats thank Turkey for support medical aid amid coronavirus pandemic via DailySabah',
 'I wonder If China were hard beaten by this virus will the world lash on China',
 'HillBeverlyhill YOU HAD FORGOTTEN TO LIST ONE OTHER ITEM TRUMPS A MURDER HAVING WELL OVER 65 000 AMERICAN CITIZENS BLOOD ON HIS HANDS TRUMP KNEW IN LATE DEC EARLY JAN ABOUT COVID19 CHOSE TO IGNORE IT HE THEN CALLED COVID19 DEMOCRATIC HOAX THEN HE LIED ABOUT COVID19 FINALLY MISLEAD US',
 'This is why we need Joe Biden GoJoe',
 'should we laugh',
 'realDonaldTrump FakeNews FB Twitter R liars R the Enemies of the Truth the American People FakeNews spew hate filled Democratic Party Plantation Propaganda Americans R being lied 2 by the FakeNews about President Trump cures 4 COVID19 Democrats Bureaucracy corruption Woke',
 'Coronavirus Outbreak LIVE Updates Gujarat reports 374 new infections 28 fatalities in 24 hours',
 'WHO US just reported deadliest day for coronavirus',
 'rindyaird These young people will likely not have severe COVID because they are young and healthy n',
 'However',
 'Here s the close up video',
 '',
 'Rscube3 Or you can say coronavirus and non coronavirus patients',
 'BRUSSELS Australia has called for an inquiry into the origin of the virus Germany and Britain are hesitating anew about inviting in the Chinese tech giant Huawei President Trump has blamed China for the contagion and is seeking to punish it',
 'Adidas is using AI and 3D printing to cope with a plunge in sales during the coronavirus pandemic via businessinsider',
 'What was your COVID19 Crisis turning point For me it s still Feb 25th CDC conf call Dr Nancy Messonnier We are asking the American public to work with us to prepare in the expectation that this could be bad It s not so much a question of if this will happen but when',
 'Itna waist of money kyo warriors to simple flowers se bhi khush hojate',
 'No thanks you can keep it',
 'It is ridiculous to say America should reopen economy go to work take Covid 19 risk when the leaders of the country are safe at home and not practicing what they are preaching',
 'Coronavirus Update San Francisco Hotels Struggling During COVID 19 Shutdown May 3 10 45 AM ET',
 'COVID19 lockdown a case study for environmental awareness YouTube',
 'Absolute fucking tosser',
 'The War Between Middle Schoolers and Their Parents Ends Now',
 'The NCLC4consumers estimates that 1 in 3 Americans have debt in collections n',
 'Federal law requires that debtors are left with at least 217 50 a week in take home pay for a family of four',
 '',
 'We love to see it',
 'realDonaldTrump Then they impeach you n',
 'Then you botch the Covid19 response',
 'Then you lose to a brain damaged Democrat',
 'Wuhan Lab released the Virus and wants the patent a cure Remdesivir Think about who is touting this drug',
 'this has been wildly derailed by coronavirus',
 'This is utter madness Who removes a lockdown at the PEAK of a pandemic Somebody who wants massive casualties and maximum harm to the economy That s who',
 'Coronavirus strikes again',
 'Hi Rich pple n',
 'Here is a life hack',
 'If you have a gateman feed him and mke sure his family hve food to stay home in lockdown Don t be enjoying lockdown and Coronavirus would be brought into ur house cause of ur cruelty',
 'What Why does he need to eat his neighbours Why has no one thought of this before tell starving people to just eat each other problem solved What could possible go wrong u200d AlexJones covid19',
 'Coronavirus Hungary Orban government muzzles journalists DW News n',
 '',
 'Coronavirus Hungary Orban government muzzles journalists DW News',
 'Rights grou

###Sentiment Analysis

#####We wanted to train the model on a constant data. As the streaming data keeps changing, we will be doing our predictions using stream data which was exported into a csv
#####The below code would predict a label as an output. This data set has two different polarities i.e. negative and positive
#####We would be predicting the outcome in the label column
#####Sampled Labeled Data is taken from Sentiment-140 Dataset

In [44]:
%sh
wget https://raw.githubusercontent.com/harshdesai7/Titanic/master/training.csv
#wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip


--2020-05-03 14:53:13-- https://raw.githubusercontent.com/harshdesai7/Titanic/master/training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6334783 (6.0M) [text/plain]
Saving to: ‘training.csv’

 0K .......... .......... .......... .......... .......... 0% 4.16M 1s
 50K .......... .......... .......... .......... .......... 1% 8.19M 1s
 100K .......... .......... .......... .......... .......... 2% 7.90M 1s
 150K .......... .......... .......... .......... .......... 3% 80.3M 1s
 200K .......... .......... .......... .......... .......... 4% 144M 1s
 250K .......... .......... .......... .......... .......... 4% 59.3M 1s
 300K .......... .......... .......... .......... .......... 5% 10.3M 1s
 350K .......... .......... .......... .......... .......... 6% 122M 0s
 400K .......... .......... .......... .......... .......... 7% 141M 0s
 450K .......... .......... .......... .......... .......... 8% 44.8M 0s
 500K .......... .......... .......... .......... .......... 8% 35.3M 0s
 550K .......... .......... .......... .......... .......... 9% 31.5M 0s
 600K .......... .......... .......... .......... .......... 10% 34.9M 0s
 650K .......... .......... .......... .......... .......... 11% 36.3M 0s
 700K .......... .......... .......... .......... .......... 12% 35.6M 0s
 750K .......... .......... .......... .......... .......... 12% 34.1M 0s
 800K .......... .......... .......... .......... .......... 13% 37.2M 0s
 850K .......... .......... .......... .......... .......... 14% 36.6M 0s
 900K .......... .......... .......... .......... .......... 15% 38.3M 0s
 950K .......... .......... .......... .......... .......... 16% 33.0M 0s
 1000K .......... .......... .......... .......... .......... 16% 39.2M 0s
 1050K .......... .......... .......... .......... .......... 17% 39.4M 0s
 1100K .......... .......... .......... .......... .......... 18% 38.2M 0s
 1150K .......... .......... .......... .......... .......... 19% 34.0M 0s
 1200K .......... .......... .......... .......... .......... 20% 38.3M 0s
 1250K .......... .......... .......... .......... .......... 21% 41.0M 0s
 1300K .......... .......... .......... .......... .......... 21% 38.0M 0s
 1350K .......... .......... .......... .......... .......... 22% 43.9M 0s
 1400K .......... .......... .......... .......... .......... 23% 71.9M 0s
 1450K .......... .......... .......... .......... .......... 24% 38.0M 0s
 1500K .......... .......... .......... .......... .......... 25% 47.4M 0s
 1550K .......... .......... .......... .......... .......... 25% 43.0M 0s
 1600K .......... .......... .......... .......... .......... 26% 110M 0s
 1650K .......... .......... .......... .......... .......... 27% 56.2M 0s
 1700K .......... .......... .......... .......... .......... 28% 38.6M 0s
 1750K .......... .......... .......... .......... .......... 29% 76.3M 0s
 1800K .......... .......... .......... .......... .......... 29% 38.5M 0s
 1850K .......... .......... .......... .......... .......... 30% 49.8M 0s
 1900K .......... .......... .......... .......... .......... 31% 59.0M 0s
 1950K .......... .......... .......... .......... .......... 32% 93.5M 0s
 2000K .......... .......... .......... .......... .......... 33% 35.6M 0s
 2050K .......... .......... .......... .......... .......... 33% 69.1M 0s
 2100K .......... .......... .......... .......... .......... 34% 98.5M 0s
 2150K .......... .......... .......... .......... .......... 35% 31.4M 0s
 2200K .......... .......... .......... .......... .......... 36% 61.7M 0s
 2250K .......... .......... .......... .......... .......... 37% 96.2M 0s
 2300K .......... .......... .......... .......... .......... 37% 153M 0s
 2350K .......... .......... .......... .......... .......... 38% 32.9M 0s
 2400K .......... .......... .......... ........

In [45]:
%sh
ls

conf
Country.csv
derby.log
eventlogs
ganglia
logs
myfile.csv
state.csv
training.csv

In [46]:
#Installing findspark library

In [47]:
%sh
pip install findspark

Collecting findspark
 Downloading https://files.pythonhosted.org/packages/b1/c8/e6e1f6a303ae5122dc28d131b5a67c5eb87cbf8f7ac5b9f87764ea1b1e1e/findspark-1.3.0-py2.py3-none-any.whl
Installing collected packages: findspark
Successfully installed findspark-1.3.0
You are using pip version 19.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [48]:
import findspark
import re
findspark.init()
import pyspark as ps
import warnings
from pyspark.sql import SQLContext

####Reading the Tweets file in Databricks

In [50]:
df = spark.read.csv('file:///databricks/driver/training.csv',header='false', inferSchema ='true')
#df = spark.read.csv('file:///databricks/driver/training.1600000.processed.noemoticon.csv',header='false', inferSchema ='true')

####Renaming Columns and Selecting only necessary Columns

In [52]:
df = df.withColumnRenamed("_c5","text")
df = df.withColumnRenamed("_c0","target")
df=df.select("text","target")

In [53]:
display(df)

text,target
"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D",0
is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!,0
@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds,0
my whole body feels itchy and like its on fire,0
"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.",0
@Kwesidei not the whole crew,0
Need a hug,0
"@LOLTrish hey long time no see! Yes.. Rains a bit ,only a bit LOL , I'm fine thanks , how's you ?",0
@Tatiana_K nope they didn't have it,0
@twittera que me muera ?,0


####Removing Null Values

In [55]:
df.count()

Out[37]: 526048

In [56]:
df = df.dropna()
df.count()

Out[38]: 20001

####Splitting data into Train and Test

In [58]:
(train_set, val_set, test_set) = df.randomSplit([0.7, 0.01, 0.29], seed = 2000)

###Fitting into a Pipeline

In [60]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier,NaiveBayes
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.ml.evaluation import RegressionEvaluator,BinaryClassificationEvaluator


tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="tf")
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5)
label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")

pipeline = Pipeline(stages=[])  # Must initialize with empty list!

basePipeline=[tokenizer, hashingTF,idf,label_stringIdx]
#basePipeline=[tokenizer, idf,label_stringIdx]

# Models to compare
#Logistic regression model
lr = LogisticRegression(maxIter=10)

pl_lr = basePipeline + [lr]
pg_lr = ParamGridBuilder()\
          .baseOn({pipeline.stages: pl_lr})\
          .addGrid(lr.regParam,[0.01, .04])\
          .addGrid(lr.elasticNetParam,[0.1, 0.4])\
          .build()

#Random forest Classifier
rf = RandomForestClassifier(numTrees=50)
pl_rf = basePipeline + [rf]
pg_rf = ParamGridBuilder()\
      .baseOn({pipeline.stages: pl_rf})\
      .build() 

#Naive Bayes Classifier
nb = NaiveBayes()
pl_nb = basePipeline + [nb]
pg_nb = ParamGridBuilder()\
      .baseOn({pipeline.stages: pl_nb})\
      .addGrid(nb.smoothing,[0.4,1.0])\
      .build()

#Combining all ParamGrids
paramGrid = pg_lr + pg_rf + pg_nb

###Cross validator for Pipeline
#####Takes 20 minutes to run

In [62]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)  # use 3+ folds in practice
cvModel = crossval.fit(train_set)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

###Best Model

In [64]:
import numpy as np
cvModel.getEstimatorParamMaps()[ np.argmax(cvModel.avgMetrics)]

Out[42]: {Param(parent='Pipeline_5a93da6c8bf3', name='stages', doc='a list of pipeline stages'): [Tokenizer_da65f882aa0b,
 HashingTF_efd33a8dfd71,
 IDF_5e3a6b2af6a1,
 StringIndexer_077f357d9c73,
 LogisticRegression_7c12a4d0e537],
 Param(parent='LogisticRegression_7c12a4d0e537', name='regParam', doc='regularization parameter (>= 0).'): 0.04,
 Param(parent='LogisticRegression_7c12a4d0e537', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1}

###Worst Model

In [66]:
cvModel.getEstimatorParamMaps()[ np.argmin(cvModel.avgMetrics)]

Out[43]: {Param(parent='Pipeline_5a93da6c8bf3', name='stages', doc='a list of pipeline stages'): [Tokenizer_da65f882aa0b,
 HashingTF_efd33a8dfd71,
 IDF_5e3a6b2af6a1,
 StringIndexer_077f357d9c73,
 NaiveBayes_a01cbd1e9ba9],
 Param(parent='NaiveBayes_a01cbd1e9ba9', name='smoothing', doc='The smoothing parameter, should be >= 0, default is 1.0'): 1.0}

###Evaluating the Model

In [68]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator()
predictions = cvModel.transform(test_set)
print("Evaluating the Model, Value found to be: ",format(evaluator.evaluate(predictions),'.2f'))

Evaluating the Model, Value found to be: 0.80

###Printing metrics

In [70]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

predictionAndLabels = predictions.select('prediction','label').rdd
metrics = BinaryClassificationMetrics(predictionAndLabels)
print("Precision Recall:", format(metrics.areaUnderPR,".2f"))
print("ROC:", format(metrics.areaUnderROC,".2f"))


Precision Recall: 0.72
ROC: 0.73

####Building the Metrics

In [72]:
import re
def paramGrid_model_name(model):
  params = [v for v in model.values() if type(v) is not list]
  name = [v[-1] for v in model.values() if type(v) is list][0]
  name = re.match(r'([a-zA-Z]*)', str(name)).groups()[0]
  return "{}{}".format(name,params)

# Resulting metric and model description
# get the measure from the CrossValidator, cvModel.avgMetrics
# get the model name & params from the paramGrid
kmeans_measures = zip(cvModel.avgMetrics, [paramGrid_model_name(m) for m in paramGrid])
metrics,model_names = zip(*kmeans_measures)

In [73]:
%sh
pip install --upgrade pip
pip install numpy

Collecting pip
 Downloading https://files.pythonhosted.org/packages/54/2e/df11ea7e23e7e761d484ed3740285a34e38548cf2bad2bed3dd5768ec8b9/pip-20.1-py2.py3-none-any.whl (1.5MB)
Installing collected packages: pip
 Found existing installation: pip 19.0.3
 Uninstalling pip-19.0.3:
 Successfully uninstalled pip-19.0.3
Successfully installed pip-20.1
Requirement already satisfied: numpy in /databricks/python3/lib/python3.7/site-packages (1.16.2)

####Plotting the Models

In [75]:
import matplotlib.pyplot as plt

plt.clf() # clear figure
fig = plt.figure( figsize=(8, 8))
plt.style.use('fivethirtyeight')
axis = fig.add_axes([0.1, 0.3, 0.8, 0.6])
# plot the metrics as Y
#plt.plot(range(len(model_names)),metrics)
plt.bar(range(len(model_names)),metrics)
# plot the model name & param as X labels
plt.xticks(range(len(model_names)), model_names, rotation=70, fontsize=14)
plt.yticks(fontsize=14)
#plt.xlabel('model',fontsize=8)
plt.ylabel('ROC AUC (better is greater)',fontsize=14)
plt.title('Model evaluations')
display(plt.show())

###Predictions
#### This is fairly good result however, there is further scope for improvement
#####0 Signifies Negative Sentiment and 1 Signifies Positive Sentiment

In [77]:
correct = predictions.where("(label = prediction)").count()
incorrect = predictions.where("(label != prediction)").count()
resultDF = sqlContext.createDataFrame([['correct', correct], ['incorrect', incorrect]], ['metric', 'value'])
display(resultDF)

metric,value
correct,4217
incorrect,1536


####Actual vs Predicted

In [79]:
counts = [predictions.where('label=1').count(), predictions.where('prediction=1').count(),
          predictions.where('label=0').count(), predictions.where('prediction=0').count()]
names = ['actual 1', 'predicted 1', 'actual 0', 'predicted 0']
display(sqlContext.createDataFrame(zip(names,counts),['Measure','Value']))

Measure,Value
actual 1,2892
predicted 1,2672
actual 0,2861
predicted 0,3081


###Word Cloud that describes sentiment

In [81]:
import numpy as np # linear algebra
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [82]:
%sh
pip install wordcloud

Collecting wordcloud
 Downloading wordcloud-1.7.0-cp37-cp37m-manylinux1_x86_64.whl (364 kB)
Requirement already satisfied: matplotlib in /databricks/python3/lib/python3.7/site-packages (from wordcloud) (3.0.3)
Requirement already satisfied: numpy>=1.6.1 in /databricks/python3/lib/python3.7/site-packages (from wordcloud) (1.16.2)
Collecting pillow
 Downloading Pillow-7.1.2-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (2.4.6)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (0.10.0)
Requirement already satisfied: python-dateutil>=2.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (2.8.0)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib->wordcloud) (1.1.0)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from cycler>=0.10->matplotlib->wordcloud) (1.12.0)
Requirement already satisfied: setuptools in /usr/lib/python3.7/site-packages (from kiwisolver>=1.0.1->matplotlib->wordcloud) (40.8.0)
Installing collected packages: pillow, wordcloud
Successfully installed pillow-7.1.2 wordcloud-1.7.0

#####Building the Text for WordCLoud

In [84]:

from pyspark.sql.functions import col
text = "" 

# iterating through list of rows 
for row in cleaned_tweets :
  for word in row :
    text = text + word 
  #print(word)
print(text)

coronavirus update in US New cases confirmed 11 new deaths 1 Total cases in US confirmed 1164007 recovered 175382 deaths 67552 0 39 daily cases increase 4577 reported today UTC timeBe that as it may the restructuring of CDC last year that eliminated positions to oversee emerging viruses in China worsened our ability to prepareTop US Somalia diplomats thank Turkey for support medical aid amid coronavirus pandemic via DailySabahI wonder If China were hard beaten by this virus will the world lash on ChinaHillBeverlyhill YOU HAD FORGOTTEN TO LIST ONE OTHER ITEM TRUMPS A MURDER HAVING WELL OVER 65 000 AMERICAN CITIZENS BLOOD ON HIS HANDS TRUMP KNEW IN LATE DEC EARLY JAN ABOUT COVID19 CHOSE TO IGNORE IT HE THEN CALLED COVID19 DEMOCRATIC HOAX THEN HE LIED ABOUT COVID19 FINALLY MISLEAD USThis is why we need Joe Biden GoJoeshould we laughrealDonaldTrump FakeNews FB Twitter R liars R the Enemies of the Truth the American People FakeNews spew hate filled Democratic Party Plantation Propaganda Americans R being lied 2 by the FakeNews about President Trump cures 4 COVID19 Democrats Bureaucracy corruption WokeCoronavirus Outbreak LIVE Updates Gujarat reports 374 new infections 28 fatalities in 24 hoursWHO US just reported deadliest day for coronavirusrindyaird These young people will likely not have severe COVID because they are young and healthy nHoweverHere s the close up videoRscube3 Or you can say coronavirus and non coronavirus patientsBRUSSELS Australia has called for an inquiry into the origin of the virus Germany and Britain are hesitating anew about inviting in the Chinese tech giant Huawei President Trump has blamed China for the contagion and is seeking to punish itAdidas is using AI and 3D printing to cope with a plunge in sales during the coronavirus pandemic via businessinsiderWhat was your COVID19 Crisis turning point For me it s still Feb 25th CDC conf call Dr Nancy Messonnier We are asking the American public to work with us to prepare in the expectation that this could be bad It s not so much a question of if this will happen but whenItna waist of money kyo warriors to simple flowers se bhi khush hojateNo thanks you can keep itIt is ridiculous to say America should reopen economy go to work take Covid 19 risk when the leaders of the country are safe at home and not practicing what they are preachingCoronavirus Update San Francisco Hotels Struggling During COVID 19 Shutdown May 3 10 45 AM ETCOVID19 lockdown a case study for environmental awareness YouTubeAbsolute fucking tosserThe War Between Middle Schoolers and Their Parents Ends NowThe NCLC4consumers estimates that 1 in 3 Americans have debt in collections nFederal law requires that debtors are left with at least 217 50 a week in take home pay for a family of fourWe love to see itrealDonaldTrump Then they impeach you nThen you botch the Covid19 responseThen you lose to a brain damaged DemocratWuhan Lab released the Virus and wants the patent a cure Remdesivir Think about who is touting this drugthis has been wildly derailed by coronavirusThis is utter madness Who removes a lockdown at the PEAK of a pandemic Somebody who wants massive casualties and maximum harm to the economy That s whoCoronavirus strikes againHi Rich pple nHere is a life hackIf you have a gateman feed him and mke sure his family hve food to stay home in lockdown Don t be enjoying lockdown and Coronavirus would be brought into ur house cause of ur crueltyWhat Why does he need to eat his neighbours Why has no one thought of this before tell starving people to just eat each other problem solved What could possible go wrong u200d AlexJones covid19Coronavirus Hungary Orban government muzzles journalists DW News nCoronavirus Hungary Orban government muzzles journalists DW NewsRights groups fear that emergency legislation enacted in Hungary to strengthen the powerYes cause your cops was beating down black ppl all over the boroughs yesterday but ain t have that same energy for the JewsDr IfeanyiUbah Man with he

In [85]:
# importing the necessery modules 
#%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt 
import csv  
 
wordcloud = WordCloud(width=4000, height=2400).generate(text) 
wordcloud = WordCloud(
        background_color='white',
        stopwords=STOPWORDS,
        width=1000,
        height=600,
        random_state=21,
        colormap='jet',
        max_words=150,
        max_font_size=200).generate(text) 

# plot the WordCloud image 
plt.figure( figsize=(18, 12))
#plt.figure() 
plt.imshow(wordcloud, interpolation="bilinear") 
plt.axis("off") 
plt.margins(x=0, y=0) 
plt.show() 

In [86]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url= "https://www.gannett-cdn.com/presto/2020/03/15/USAT/e6e30693-9224-4c89-b003-6cc4b4348528-insta-noemoji-heart.png?width=660&height=660&fit=crop&format=pjpg&auto=webp")

In [87]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url= "https://upload.wikimedia.org/wikipedia/commons/e/ea/Thats_all_folks.svg")